In [ ]:
# default_exp files

# files
> Easily upload, download, and manage files on Server.

In [ ]:
# export
import requests

from yx_motor.api import API


class Files:
    "Class for handling Files endpoints"

    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = "files/"

    def download_file(self, file_uuid: str, download_path: str, version: int = None):
        response = self.api.get(
            url=f'{self.base_endpoint}content', params={"id": file_uuid,"version": version}
        )
        with open(download_path, 'wb') as f:
            f.write(response.content)
        return response

    def upload_file(
        self,
        filename: str,
        upload_path: str,
        description: str = "",
        conflict_action=None,
    ):
        upload_headers = {
            "Content-Type": "application/json",
            "Accept": "*/*",
            "Accept-Language": "en-US,en;q=0.5",
            "Accept-Encoding": "gzip,deflate",
            "path": f"{upload_path}",
            "description": f"{description}",
            "conflict_action": f"{conflict_action}",
        }

        with open(filename, "rb") as f:
            blob = f.read()
        response = self.api.post(url=f"{self.base_endpoint}", data=blob, non_default_headers=upload_headers)
        return response

    def update_file(self):
        # This seems to update metainfo, may not be needed for MVP
        # TODO: MVP
        pass

    def get_file_versions(self, file_uuid: str):
        response = self.api.get(url=f"{self.base_endpoint}versions/", params={"id": file_uuid})
        return response

    def delete_file(self, asset_path: str, hard=False):
        payload = {"assetPaths": [f"{asset_path}"]}

        if hard == True:
            targeturl = "remove"
        else:
            targeturl = "softDelete"

        response = self.api.post(url=f'{self.base_endpoint}{targeturl}', data=payload)

        return response

    def move_file(
        self,
        source_path: str,
        target_path: str,
        move_type="move",
        versions_action="ALL_VERSIONS",
        conflicts_action="SKIP",
    ):
        # TODO discuss move_type, move or copy
        payload = {
            "assets": [
                {"sourcePath": f"{source_path}", "targetPath": f"{target_path}"}
            ],
            "targetPath": "string",
            "versionsAction": f"{versions_action}",
            "conflictsAction": f"{conflicts_action}",
        }

        response = self.api.post(url=f'{self.base_endpoint}{move_type}', data=payload)
        return response

    def restore_deleted_file(self, asset_path: str = None, asset_id: str = None):
        asset_paths = {}
        asset_ids = {}
        if asset_path:
            asset_paths = {"assetPaths": [f"{asset_path}"]}
        if asset_id:
            asset_ids = {"assetIds": [f"{asset_id}"]}
        payload = {**asset_paths, **asset_ids, **{"onlyDescendants": True}}
        response = self.api.post(url=f"{self.base_endpoint}restoreDeleted", data=payload)
        return response

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted 06_workflows.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.


In [ ]:
#hide
from nbdev.sync import script2notebook
#script2notebook()

In [ ]:
#hide
# just removing the insecure warning for now
# TODO: Secure requests and remove this code
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
from yx_motor.client import Client

In [ ]:
#hide
from private import server_vars

In [ ]:
#hide
base_url=server_vars['base_url']
login_email = server_vars['login_email']
login_pwd = server_vars['login_pwd']

In [ ]:
#hide
dev_client = Client(base_url,
              login_email, 
              login_pwd)

In [ ]:
dev_client.api.is_authenticated
files = Files(dev_client.api)

In [ ]:
response = files.download_file("68efa737-d9e0-4a91-8379-737062ddc158","test.log")

In [ ]:
response.content

b'-|worker|Retrieved file example.yxzp\r\n-|worker|Engine run started\r\n2|1|Autodetected Code Page: UTF-8\r\n2|8|input_one.csv|4 records were read from "input_one.csv"\r\n3|1|Autodetected Code Page: UTF-8\r\n3|8|input_two.csv|1 records were read from "input_two.csv" (ended by a downstream tool)\r\n-|worker|Engine run finished in 0.47 seconds'

In [ ]:
response

<Response [404]>

In [ ]:
response.request.method

'GET'